## Importando módulos de la Enaho-INEI
Se va a importar la información de la encuesta Encuesta Nacional de los Hogares (Enaho) realizado por Inei que se encuentra en sus microdatos http://iinei.inei.gob.pe/microdatos/. Este código es una adaptación y extensión de uno creado y presentado en una clase por Angel Guillen y Erick Oré

In [1]:
import pandas as pd
import requests
import time
from zipfile import ZipFile, BadZipfile
import os
from os.path import join
from os import chdir, listdir, rmdir
from shutil import move

Se define todas las Enahos anuales

In [2]:
Enaho_anual = {
    2004: "280", #0
    2005: "281", #1
    2006: "282", #2
    2007: "283", #3 
    2008: "284", #4
    2009: "285", #5
    2010: "279", #6
    2011: "291", #7
    2012: "324", #8
    2013: "404", #9
    2014: "440", #10
    2015: "498", #11
    2016: "546", #12
    2017: "603", #13
    2018: "634", #14
    2019: "687", #15
    2020: "737", #16
    2021: "759" #17
}

Se busca importar algunos módulos en particular. Los más usuales son los siguientes:

In [3]:
Modulos = ["02", "03", "05"]

Donde el módulo 02 representa las característica de los miembros del hogar, el módulo 03 la educación y el 05 los ingresos.

### Importando las Enaho

Primero, debido a que no siempre es necesario usar todas las Enahos, se puede seleccionar un set. Esto también ayudará a que la importación de los datos sea más rápida. Se define como set de las `Enaho anuales` al periodo `2017-2021`

In [4]:
FechaIni = 2019
FechaFin = 2019

Rango = list(range(FechaIni, FechaFin+1))
Enaho_anual = {key: Enaho_anual[key] for key in Rango}

Por corroborar, las siguientes deberían dar los resultados deseados

In [5]:
Enaho_anual

{2019: '687'}

Se separá los `keys` y los `values`

In [6]:
Periodos = list(Enaho_anual.keys())
Encuestas = list(Enaho_anual.values())

La importación y presentación de los archivos `.dta` se puede realizar de dos maneras diferentes: (1) tener todos los módulos en cada carpeta correspondiente a su año, (2) tener todos los módulos en una única carpeta.

El código mostrado a continuación corresponde al punto (1), donde el resultado debería ser muchas carpetas, donde cada una representa un año y contiene los módulos seleccionados

In [ ]:
periodo = Periodos[0]


for i, n in enumerate(Encuestas):
    
    
    # Creando la carpeta por cada periodo
    try:
        os.makedirs(f"Bases de datos//{str(periodo)}")
    except:
        pass
    
    
    # Iterando cada módulo para cada periodo
    for m in Modulos:
        
        # Haciendo el request con la página de INEI
        try:
            r = requests.get(f"http://iinei.inei.gob.pe/iinei/srienaho/descarga/STATA/{n}-Modulo{m}.zip", allow_redirects=True)
        except:
            time.sleep(3)
            r = requests.get(f"http://iinei.inei.gob.pe/iinei/srienaho/descarga/STATA/{n}-Modulo{m}.zip", allow_redirects=True)
        
        # Abriendo el Zip y renombrándolo
        open(f"Bases de datos//{str(periodo)}//Modulo{n}{m}.zip", 'wb').write(r.content)

        # Extrayendo todos los archivos del Zip
        with ZipFile(f"Bases de datos//{str(periodo)}//Modulo{n}{m}.zip", 'r') as file:
            file.extractall(path = f"Bases de datos//{str(periodo)}")

        try:
            # Moviendo los archivos a la ruta
            ruta = join(f"Bases de datos//{str(periodo)}", f"{n}-Modulo{m}")
            
            for filename in listdir(ruta):
                move(join(ruta, filename), join(f"Bases de datos//{str(periodo)}", filename))
                        
            # Eliminando archivos no usados
            rmdir(ruta)
            
        except:
            pass
        
        
        # Eliminando todos los archivos que no son de interés
        for file in listdir(f"Bases de datos//{str(periodo)}"):
            if not file.startswith("enaho01"):
                os.remove(f"Bases de datos//{str(periodo)}//{file}")
            if file.endswith("a.dta"):
                os.remove(f"Bases de datos//{str(periodo)}//{file}")
                            
    periodo += 1

El siguiente código es el punto (2) donde todos los módulos de cada año estarán en una única carpeta

In [7]:
for i, n in enumerate(Encuestas):
    
    
    # Creando la carpeta por cada periodo
    try:
        os.makedirs("Bases de datos")
    except:
        pass
    
    
    # Iterando cada módulo para cada periodo
    for m in Modulos:
        
        # Haciendo el request con la página de INEI
        try:
            r = requests.get(f"http://iinei.inei.gob.pe/iinei/srienaho/descarga/STATA/{n}-Modulo{m}.zip", allow_redirects=True)
        except:
            time.sleep(3)
            r = requests.get(f"http://iinei.inei.gob.pe/iinei/srienaho/descarga/STATA/{n}-Modulo{m}.zip", allow_redirects=True)
        
        # Abriendo el Zip y renombrándolo
        open(f"Bases de datos//Modulo{n}{m}.zip", 'wb').write(r.content)

        # Extrayendo todos los archivos del Zip
        with ZipFile(f"Bases de datos//Modulo{n}{m}.zip", 'r') as file:
            file.extractall(path = "Bases de datos")

        try:
            # Moviendo los archivos a la ruta
            ruta = join("Bases de datos", f"{n}-Modulo{m}")
            
            for filename in listdir(ruta):
                move(join(ruta, filename), join("Bases de datos", filename))
                        
            # Eliminando archivos no usados
            rmdir(ruta)
            
        except:
            pass
        
        
        # Eliminando todos los archivos que no son de interés
        for file in listdir("Bases de datos"):
            if not file.startswith("enaho01"):
                os.remove(f"Bases de datos//{file}")
            if file.endswith("a.dta"):
                os.remove(f"Bases de datos//{file}")